In [19]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.io as pio
# Constants
API_KEY = ''  # Replace with your actual API key
SINCE_DATE = int(datetime(2020, 1, 1).timestamp())  # Jan 1, 2015
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/fees/volume_sum',
    'https://api.glassnode.com/v1/metrics/addresses/active_count',
    'https://api.glassnode.com/v1/metrics/transactions/transfers_volume_entity_adjusted_sum'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

In [20]:
def calculate_smoothed_bands(df, column='volume_sum', smoothing_fun='sma', smoothing_window=7, 
                         range_method='maxmin', range_window=90):
   # Calculate smoothed version based on method
   if smoothing_fun == 'sma':
       smoothed_raw = df[column].rolling(window=smoothing_window).mean()
   elif smoothing_fun == 'ema':
       smoothed_raw = df[column].ewm(span=smoothing_window).mean()
   elif smoothing_fun == 'med':
       smoothed_raw = df[column].rolling(window=smoothing_window).median()
   else:
       raise ValueError("Invalid smoothing method. Choose 'sma', 'ema', or 'med'.")
   
   # Calculate bands based on smoothed data
   if range_method == 'maxmin':
       upper_band = smoothed_raw.rolling(window=range_window).max()
       lower_band = smoothed_raw.rolling(window=range_window).min()
   
   elif range_method == 'zscore':
       mean = smoothed_raw.rolling(window=range_window).mean()
       std = smoothed_raw.rolling(window=range_window).std()
       upper_band = mean + std
       lower_band = mean - std
   
   elif range_method == 'percentile':
       upper_band = smoothed_raw.rolling(window=range_window).quantile(0.95)
       lower_band = smoothed_raw.rolling(window=range_window).quantile(0.05)
   
   elif range_method == 'meandev':
       mean = smoothed_raw.rolling(window=range_window).mean()
       mean_deviation = (smoothed_raw - mean).abs().rolling(window=range_window).mean()
       upper_band = mean + mean_deviation
       lower_band = mean - mean_deviation
   
   else:
       raise ValueError("Invalid range method. Choose 'maxmin', 'zscore', 'percentile', or 'meandev'.")
   
   return smoothed_raw, upper_band, lower_band

In [21]:
# Apply the functions to the merged dataframe with normalization
merged_df['fee_ema_percentile'], merged_df['fee_upper'], merged_df['fee_lower']= calculate_smoothed_bands(merged_df, column='volume_sum',smoothing_fun='ema', smoothing_window=7, 
                         range_method='percentile', range_window=90)
merged_df['volume_sma_zscore'], merged_df['volume_upper'], merged_df['volume_lower']= calculate_smoothed_bands(merged_df, column='transfers_volume_entity_adjusted_sum',smoothing_fun='sma', smoothing_window=7, 
                         range_method='zscore', range_window=180)
merged_df['address_sma_meandev'], merged_df['address_upper'], merged_df['address_lower']= calculate_smoothed_bands(merged_df, column='active_count',smoothing_fun='sma', smoothing_window=7, 
                         range_method='meandev', range_window=180 )


In [22]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

def create_indicator_chart(merged_df, indicator_column, chart_title, bands="on", upper_lower=["fee_upper", "fee_lower"]):
   # Filter data for the last year
   one_year_ago = datetime.now() - timedelta(days=730)
   merged_df_last_year = merged_df[merged_df.index > one_year_ago]

   # Define colors
   GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
   BLUE_COLOR = 'rgb(0, 0, 255)'  # Blue
   GREEN_COLOR = 'rgb(0, 255, 0)'  # Green for upper band
   RED_COLOR = 'rgb(255, 0, 0)'    # Red for lower band

   # Create the visualization
   fig = make_subplots(specs=[[{"secondary_y": True}]])

   # Add price trace
   fig.add_trace(
       go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], 
                 name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
       secondary_y=False,
   )

   # Add indicator trace
   indicator = merged_df_last_year[indicator_column]
   fig.add_trace(
       go.Scatter(
           x=merged_df_last_year.index,
           y=indicator,
           name=indicator_column,
           line=dict(color=BLUE_COLOR, width=2),
           mode='lines'
       ),
       secondary_y=True,
   )

   # Add band traces if enabled
   if bands == "on":
       # Simply use the provided column names directly
       fig.add_trace(
           go.Scatter(
               x=merged_df_last_year.index,
               y=merged_df_last_year[upper_lower[0]], 
               name="Upper Band",
               line=dict(color=GREEN_COLOR, width=2, dash='dash'),  # Increased width to 2
               mode='lines'
           ),
           secondary_y=True,
       )
       fig.add_trace(
           go.Scatter(
               x=merged_df_last_year.index,
               y=merged_df_last_year[upper_lower[1]], 
               name="Lower Band",
               line=dict(color=RED_COLOR, width=2, dash='dash'),  # Increased width to 2
               mode='lines'
           ),
           secondary_y=True,
       )

   # Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
   for month in [1, 4, 7, 10]:
       for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
           date = pd.Timestamp(year=year, month=month, day=1)
           if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
               fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

   # Get the last value of the indicator and format it
   last_value = indicator.iloc[-1]
   
   # Function to format number to K, M, B
   def format_number(num):
       if abs(num) >= 1e9:
           return f"{num/1e9:.2f}B"
       elif abs(num) >= 1e6:
           return f"{num/1e6:.2f}M"
       elif abs(num) >= 1e3:
           return f"{num/1e3:.2f}K"
       else:
           return f"{num:.2f}"
   
   formatted_value = format_number(last_value)

   # Add annotation for the last value
   fig.add_annotation(
       x=0.95,  
       y=last_value*0.88,  
       xref="paper",
       yref="y2", 
       text=formatted_value,
       showarrow=False,
       font=dict(size=18, color=BLUE_COLOR),
       align="left",
       xanchor="left",
       yanchor="middle",
   )

   # Update layout
   fig.update_layout(
       title={
           'text': chart_title,
           'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
       },
       showlegend=False,  
       hovermode="x unified",
       plot_bgcolor='white',
       paper_bgcolor='white',
       font={'color': 'black', 'size': 14},
       width=1000,  
       height=450,  
   )

   # Update axes
   fig.update_xaxes(
       showgrid=False, 
       tickfont={'color': 'black', 'size': 14},
       zeroline=False,
       title_text=''  
   )
   fig.update_yaxes(
       showgrid=False, 
       secondary_y=False, 
       tickfont={'color': GREY_COLOR, 'size': 14},
       zeroline=False,
       showline=True,
       linecolor=GREY_COLOR,
       ticks='outside',
       tickcolor=GREY_COLOR,
       title_text='',
       title_font=dict(size=18)
   )
   fig.update_yaxes(
       showgrid=False, 
       secondary_y=True, 
       tickfont={'color': GREY_COLOR, 'size': 14},
       zeroline=False,
       showline=True,
       linecolor=GREY_COLOR,
       ticks='outside',
       side='right',
       tickcolor=GREY_COLOR,
       title_text='',
       title_font=dict(size=18),
       autorange=True
   )

   return fig

In [23]:
# Create charts for each indicator
indicators = [
    ('fee_ema_percentile',"Bitcoin: Total Fee Volume", "on", ["fee_upper", "fee_lower"]),
    ('volume_sma_zscore',"Bitcoin: Entity-Adjusted Volume", "on", ["volume_upper", "volume_lower"]),
    ('address_sma_meandev',"Bitcoin: Active Address", "on", ["address_upper", "address_lower"])
]

for indicator, title,  bands, upper_lower in indicators:
    fig = create_indicator_chart(merged_df, indicator, title, bands, upper_lower)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_{indicator}_two_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [24]:
merged_df.tail()

,price_usd_close,volume_sum,active_count,transfers_volume_entity_adjusted_sum,fee_ema_percentile,fee_upper,fee_lower,volume_sma_zscore,volume_upper,volume_lower,address_sma_meandev,address_upper,address_lower
t,,,,,,,,,,,,,
2024-11-27,95981.580554,9.464919e+05,814017,1.435566e+10,1.353688e+06,2.089333e+06,419147.979464,1.977407e+10,1.129380e+10,4.687355e+09,872571.714286,795879.381217,642469.880688
2024-11-28,95651.196450,1.312534e+06,748444,1.358155e+10,1.343400e+06,2.089333e+06,419147.979464,1.760666e+10,1.143204e+10,4.680076e+09,840737.428571,796516.183995,644386.304894
2024-11-29,97447.132368,1.266053e+06,775525,1.369030e+10,1.324063e+06,2.089333e+06,419147.979464,1.567708e+10,1.153037e+10,4.693847e+09,824059.000000,796971.735317,646296.601190
2024-11-30,96514.052710,9.796642e+05,827501,5.965496e+09,1.237963e+06,2.089333e+06,419147.979464,1.447432e+10,1.160592e+10,4.709897e+09,816149.000000,797343.453302,648094.692729
2024-12-01,97311.455238,1.963117e+06,725499,5.608371e+09,1.419252e+06,2.089333e+06,419147.979464,1.393149e+10,1.167136e+10,4.725939e+09,804422.428571,797589.042271,649888.030745


In [25]:
merged_df.to_csv('bitcoin_analysis_data_fundamental2.csv', index=True)

In [26]:
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Change the grey color to a darker shade
GREY_COLOR = 'rgba(70, 70, 70, 0.9)'  # Darker, more opaque grey

def normalize_value(value, min_val, max_val):
    range_val = max_val - min_val
    if range_val == 0:
        return 0
    normalized = (value - min_val) / range_val * 200 - 100
    return normalized

def format_number(number):
    """Format number with k, M, B suffixes"""
    abs_number = abs(number)
    if abs_number >= 1e9:
        return f"{number/1e9:.2f}B"
    elif abs_number >= 1e6:
        return f"{number/1e6:.2f}M"
    elif abs_number >= 1e3:
        return f"{number/1e3:.2f}k"
    else:
        return f"{number:.2f}"

def create_dot_plot(df, metrics_titles, days=90):
    start_date = df.index[-1] - timedelta(days=days)
    df_filtered = df[df.index >= start_date]
    
    fig = go.Figure()
    y_values = []
    
    for metric, title in metrics_titles.items():
        min_val = df_filtered[metric].min()
        max_val = df_filtered[metric].max()
        current_val = df_filtered[metric].iloc[-1]
        
        # Normalize values
        norm_min = -100
        norm_max = 100
        norm_current = normalize_value(current_val, min_val, max_val)
        
        y_values.append(title)
        
        # Add grey line (darker)
        fig.add_trace(go.Scatter(
            x=[norm_min, norm_max], y=[title, title], mode='lines',
            line=dict(color=GREY_COLOR, width=3),  # Using darker grey
            showlegend=False
        ))
        
        # Add min value (red filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_min], y=[title], mode='markers+text',
            marker=dict(color='red', size=16, symbol='circle'),
            text=[format_number(min_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Min'
        ))
        
        # Add max value (green filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_max], y=[title], mode='markers+text',
            marker=dict(color='green', size=16, symbol='circle'),
            text=[format_number(max_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Max'
        ))
        
        # Add current value (grey halo dot)
        fig.add_trace(go.Scatter(
            x=[norm_current], y=[title], mode='markers+text',
            marker=dict(
                color='white',
                size=16,
                line=dict(color=GREY_COLOR, width=3),  # Using darker grey
                symbol='circle'
            ),
            text=[format_number(current_val)],
            textposition="top center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Current'
        ))
    
    # Add vertical dashed lines at -50%, 0%, and 50%
    for value in [-50, 0, 50]:
        fig.add_shape(
            type="line",
            x0=value, x1=value, y0=0, y1=1,
            yref="paper",
            xref="x",
            line=dict(color=GREY_COLOR, width=1, dash="dash")  # Using darker grey
        )
    
    fig.update_layout(
        title={
            'text': f'Metric Values Over Last {days} Days',
            'font': {'color': GREY_COLOR, 'size': 20},  # Using darker grey
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title={
            'text': 'Normalized Value (%)',
            'font': {'color': GREY_COLOR, 'size': 16}  # Using darker grey
        },
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(y_values))),
            ticktext=y_values,
            tickfont={'color': GREY_COLOR, 'size': 16},  # Using darker grey
            side='left',
            title_standoff=35
        ),
        height=400 + (len(metrics_titles) * 30),
        width=1200,
        showlegend=False,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=100, r=100, t=50, b=20),
        font=dict(color=GREY_COLOR, size=14)  # Using darker grey
    )
    
    fig.update_xaxes(
        showgrid=False, 
        range=[-110, 110],
        tickvals=[-100, -50, 0, 50, 100],
        ticktext=['-100%<br>MIN-90D', '-50%', '0%', '50%', '100%<br>MAX-90D'],
        tickfont={'color': GREY_COLOR, 'size': 16},  # Using darker grey
        showline=False,
        zeroline=False
    )
    fig.update_yaxes(showgrid=False)
    
    return fig

In [27]:

# Manually define metrics and their titles
metrics_titles = {
    'volume_sum':' Total Fee Volume',
    'transfers_volume_entity_adjusted_sum':'Entity-Adjusted Volume',
    'active_count':'Active Address'
    
    # Add more metrics and titles as needed
}

# Create the dot plot
fig = create_dot_plot(merged_df, metrics_titles)

# Show the plot
fig.show()

# Optionally, save the plot as an HTML file
import plotly.io as pio
pio.write_html(fig, file='normalized_fundamental2_raw_metrics_dot_plot.html')